In [ ]:
import numpy as np

TEST_MODE = False
TUNING_MODE = False
WHERE_PLACE = "kaggle" #The WHERE_PLACE variable must have the following value:"local","kaggle","colab"
Phase_1 = False
Phase_2 = False
NAME_OUTPUT_FILE = 'A_TBCM-Stacking_v18'
TARGET = 'target'

# Range of values for selection in phase 1 :
N_ESTIMATORS_RANG = list(range(3000,4000,100))# [500] #
RANG_LEARNING_RATE = list(np.arange(0.001,0.011,0.002)) #list(np.arange(5e-3,1e-1,5e-3)) # [0.005]

CV =  5 # 
CV_O =  5 # 
TEST_SIZE = 0.02

N_TRIALS = 50
SEED = 259
GPU = False

SELECT_PRED = False
SELECT_PRED2 = False

HIGH_PARAM_END = False

# ------------------------------------------------------------------------------
# Path definition
# ------------------------------------------------------------------------------
from pathlib import Path
DATA_PATH_Kaggle = Path("../input/tabular-playground-series-mar-2021")
DATA_PATH_Colab = Path("/content/drive/My Drive/Colab Notebooks/TB1")
LOG_PATH = Path("./log/")
NAME_RUN_LOG = 'Run__Logs.log'

In [ ]:
####### Берем полученные наилучшие значения гиперпараметров с прошлых итераций
# Значения получены за несколько полных прогонов в ноутбуке - A_LGBM-tuning(7) :
final_estimator_params = {
'n_estimators': 20000, ######
'learning_rate': 0.0021,#########
'num_leaves': 104, ############
'max_bin': 152, ##########
'bagging_fraction': 0.5222799128093439,#######
'feature_fraction': 0.11749789057773732,#######
'bagging_freq': 56, ##########
'metric' : 'auc', ###########
}

####### Берем полученные наилучшие значения гиперпараметров для модели 1 (LGBMClassifier)
model_1_params = {
'n_estimators': 20000, ######
'learning_rate': 0.0021,#########
'num_leaves': 104, ############
'max_bin': 152, ##########
'bagging_fraction': 0.5222799128093439,#######
'feature_fraction': 0.11749789057773732,#######
'bagging_freq': 56, ##########
'metric' : 'auc', ###########
}

In [ ]:
####### Берем полученные наилучшие значения гиперпараметров для модели 2 (XGBClassifier)
model_2_params = {
    'n_estimators': 3500,
    'learning_rate': 0.003,   
    'max_depth': 17, 
    'min_child_weight': 10, 
    'colsample_bytree': 0.2975589566443808, 
    'subsample': 0.9573469985511692, 
    'lambda': 0.31928467435687996, 
    'alpha': 0.6847713149218315, 
    'gamma': 0.5482242693082249
}

####### Берем полученные наилучшие значения гиперпараметров для модели 3 (CATBOOST)
model_3_params = {  
    "iterations" : 12000,
    "learning_rate" : 0.012,
    'depth': 9, 
    'border_count': 66, 
    'bagging_temperature': 0.8368744913937998,
    'random_strength': 0.6663751462690772, 
    'l2_leaf_reg': 22,
    "loss_function" : 'Logloss',
    "eval_metric" : 'AUC',
    "simple_ctr" : 'Counter:CtrBorderCount=30',
    "logging_level" : 'Silent',
    "boosting_type" : 'Plain',
    "random_state" : SEED
}

In [ ]:
# загружаем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from lightgbm import LGBMClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import RandomSampler
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, UniformDistribution
# импортируем класс StandardScaler, позволяющий
# выполнить стандартизацию
from sklearn.preprocessing import StandardScaler, LabelEncoder
from category_encoders import TargetEncoder, OrdinalEncoder
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

In [ ]:
from time import time
import datetime
start_time = time()

if TEST_MODE:
    final_estimator_params.update({'n_estimators': 500})
    NAME_OUTPUT_FILE = '_TEST_' + NAME_OUTPUT_FILE + '_TEST_'
    print('**************************************************************')
    print('*** TEST mode ***')
    print('**************************************************************')
    
if WHERE_PLACE == "local":
    input_path = Path('Data')
    d_ext = 'csv'
    print("LOCAL")
elif  WHERE_PLACE == "kaggle":
    input_path = DATA_PATH_Kaggle
    d_ext = 'csv'
    print("KAGGLE")
elif  WHERE_PLACE == "colab":
    from google.colab import drive
    drive.mount('/content/drive/')
    #!ls -al "/content/drive/My Drive/Colab Notebooks/TB1"
    input_path = DATA_PATH_Colab
    d_ext = 'dcv'
    !pip install catboost
    !pip install optuna
    !pip install category_encoders
    print("Google Colab")
else:
    raise Exception('the WHERE_PLACE variable must have the following value: "local, kaggle, colab".')
    
if GPU:
    params['device'] = 'gpu'
    params['gpu_platform_id'] = 0
    params['gpu_device_id'] = 0
    !pip uninstall lightgbm -y
    # install lightgbm GPU
    !pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"
    
LOG_PATH.mkdir(parents=True, exist_ok=True)
with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
    print('\n\n***********************\n***********************', 
          file=logfile)
    print('Время старта скрипта: {}'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)
print('Время старта скрипта: {}'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")))

In [ ]:
# увеличиваем количество выводимых столбцов
pd.set_option('display.max_columns', 60)
# загружаем набор
data =  pd.read_csv(input_path / ('train.' + d_ext), index_col='id')
data.head()

In [ ]:
if TEST_MODE:
    data = data[:1000]
    print("TEST MODE, Data shape = ",data.shape)
# selection of predictors
if SELECT_PRED:
    # Separately, we selected the significant priediktors
    # https://www.kaggle.com/c/tabular-playground-series-mar-2021/discussion/224530
    # Variant 1:
    SELECTED_FEATURES = [ # Noisy Cat Variables - 'cat5', 'cat7', 'cat8', 'cat10'
                        'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat6', 'cat9', 'cat11','cat12','cat13',
                        'cat14','cat15','cat16','cat17','cat18','cont0','cont1','cont2','cont3','cont5', 
                        'cont6', 'cont7', 'cont8', 'cont9', 'cont10']

# selection of predictors
if SELECT_PRED2:
    # Separately, we selected the significant priediktors by the BoostARoota method
    # Here they are, Variant 2 :
    SELECTED_FEATURES = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 
                         'cat12','cat13','cat14','cat15','cat16','cat17','cat18','cont0','cont1','cont2','cont3',#'cont4',
                         'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10']
    
if SELECT_PRED or SELECT_PRED2:
    selfeat = SELECTED_FEATURES.copy()
    selfeat.append(TARGET)
    data = data[selfeat]
    print(data.shape)

In [ ]:
if TUNING_MODE:
    # создаем обучающий массив признаков, обучающий массив меток,
    # тестовый массив признаков, тестовый массив меток
    train, test, y_train, y_test = train_test_split(data.drop(TARGET, axis=1), 
                                                    data[TARGET], 
                                                    test_size=TEST_SIZE, 
                                                    stratify=data[TARGET], 
                                                    random_state=SEED)
else:
    # загружаем тестовый  набор
    test =  pd.read_csv(input_path / ('test.' + d_ext), index_col='id')
    # If selection of predictors
    if SELECT_PRED or SELECT_PRED2:
        test = test[SELECTED_FEATURES]

    # формируем массив меток и массив признаков
    train = data.drop(TARGET, axis=1)              
    y_train = data[TARGET] 

In [ ]:
train

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
if TUNING_MODE:
    y_test.reset_index(drop=True, inplace=True)


In [ ]:
# создаем список количественных признаков
num_columns = train.select_dtypes(exclude='object').columns.tolist()
# создаем списки категориальных признаков
cat_columns = data.select_dtypes(include='object').columns

In [ ]:
%%time

if TUNING_MODE:
    # создаем конвейеры финальной модели 
    # трансформеры
    cat_pipe_final = Pipeline([('enc', OneHotEncoder(sparse=False, handle_unknown='ignore'))])
    num_pipe_final = Pipeline([('scal', None)]) #
    # создаем список трехэлементных кортежей с преобразованиями для определенного типа признаков
    transformers_final = [('numf', num_pipe_final, num_columns),
                          ('catf', cat_pipe_final, cat_columns)]
    # передаем список трансформеров в ColumnTransformer
    transformer_final = ColumnTransformer(transformers=transformers_final)
    # задаем итоговый конвейер
    ml_pipe_final = Pipeline([('trf', transformer_final), 
                            ('mlf', LGBMClassifier(**final_estimator_params))])

    # обучаем обновленную модель на всей обучающей выборке
    ml_pipe_final.fit(train, y_train)
    print('AUC на тестовой выборке: {}'.format(
            roc_auc_score(y_test, ml_pipe_final.predict_proba(test)[:, 1])))

In [ ]:
# создаем конвейеры модели 1
cat_pipe_1 = Pipeline([('enc', OneHotEncoder(sparse=False, handle_unknown='ignore'))])
num_pipe_1 = Pipeline([('scal', None)]) #
# создаем список трехэлементных кортежей с преобразованиями для определенного типа признаков
transformers_1 = [('num1', num_pipe_1, num_columns),
                  ('cat1', cat_pipe_1, cat_columns)]
# передаем список трансформеров в ColumnTransformer
transformer_1 = ColumnTransformer(transformers=transformers_1)
# задаем итоговый конвейер
ml_pipe_1 = Pipeline([('tr1', transformer_1), 
                        ('ml1', LGBMClassifier(**model_1_params))])

In [ ]:
# создаем конвейеры модели 2
cat_pipe_2 = Pipeline([('enc', TargetEncoder() )])
num_pipe_2 = Pipeline([('scal', None)]) 
# создаем список трехэлементных кортежей с преобразованиями для определенного типа признаков
transformers_2 = [('num2', num_pipe_2, num_columns),
                  ('cat2', cat_pipe_2, cat_columns)]
# передаем список трансформеров в ColumnTransformer
transformer_2 = ColumnTransformer(transformers=transformers_2)
# задаем итоговый конвейер
ml_pipe_2 = Pipeline([('tr2', transformer_2), 
                        ('ml2', xgb.XGBClassifier(**model_2_params) )])

In [ ]:
# создаем конвейеры модели 3
cat_pipe_3 = Pipeline([('enc', OneHotEncoder(sparse=False, handle_unknown='ignore') )])
num_pipe_3 = Pipeline([('scal', None)]) 
# создаем список трехэлементных кортежей с преобразованиями для определенного типа признаков
transformers_3 = [('num3', num_pipe_3, num_columns),
                  ('cat3', cat_pipe_3, cat_columns)]
# передаем список трансформеров в ColumnTransformer
transformer_3 = ColumnTransformer(transformers=transformers_3)
# задаем итоговый конвейер
ml_pipe_3 = Pipeline([('tr3', transformer_3), 
                        ('ml3', CatBoostClassifier(**model_3_params) )])

In [ ]:
# создаем конвейеры модели 4
cat_pipe_4 = Pipeline([('enc', OrdinalEncoder()  )]) #  TargetEncoder()
num_pipe_4 = Pipeline([('scal', StandardScaler())]) 
# создаем список трехэлементных кортежей с преобразованиями для определенного типа признаков
transformers_4 = [('num4', num_pipe_4, num_columns),
                  ('cat4', cat_pipe_4, cat_columns)]
# передаем список трансформеров в ColumnTransformer
transformer_4 = ColumnTransformer(transformers=transformers_4)
# задаем итоговый конвейер
ml_pipe_4 = Pipeline([('tr4', transformer_4), 
                        ('ml4', LogisticRegression(C=4.122, class_weight='balanced', dual=False,
                                       fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                                       max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                                       random_state=259, solver='lbfgs', tol=0.0001, verbose=0,
                                       warm_start=False)
                         )])

In [ ]:
# получаем ответы (вероятности) для проверочных блоков перекрестной проверки
m_1_probs = cross_val_predict(ml_pipe_1, train, y_train, cv=5, n_jobs=-1, method='predict_proba')[:,1]

In [ ]:
if TUNING_MODE:
    print(roc_auc_score(y_train, m_1_probs ))

In [ ]:
m_2_probs = cross_val_predict(ml_pipe_2, train, y_train, cv=5, n_jobs=-1, method='predict_proba')[:,1]

In [ ]:
if TUNING_MODE:
    print(roc_auc_score(y_train, m_2_probs ))

In [ ]:
m_3_probs = cross_val_predict(ml_pipe_3, train, y_train, cv=5, n_jobs=-1, method='predict_proba')[:,1]

In [ ]:
if TUNING_MODE:
    print(roc_auc_score(y_train, m_3_probs ))

In [ ]:
m_4_probs = cross_val_predict(ml_pipe_4, train, y_train, cv=5, n_jobs=-1, method='predict_proba')[:,1]

In [ ]:
if TUNING_MODE:
    print(roc_auc_score(y_train, m_4_probs ))

In [ ]:
m_1_probs = pd.DataFrame(m_1_probs,columns=['m_1_probs'])
m_2_probs = pd.DataFrame(m_2_probs,columns=['m_2_probs'])
m_3_probs = pd.DataFrame(m_3_probs,columns=['m_3_probs'])
m_4_probs = pd.DataFrame(m_4_probs,columns=['m_4_probs'])

#m_1_probs.index  = train.index
#m_2_probs.index  = train.index
#m_3_probs.index  = train.index
#m_4_probs.index  = train.index

In [ ]:
m_1_probs

In [ ]:
m_1_probs.to_csv('m_1_probs_train_1.'+d_ext)
m_2_probs.to_csv('m_2_probs_train_1.'+d_ext)
m_3_probs.to_csv('m_3_probs_train_1.'+d_ext)
m_4_probs.to_csv('m_4_probs_train_1.'+d_ext)

In [ ]:
#m_1_probs =  pd.read_csv(input_path / ('m_1_probs.'+d_ext), index_col='id')

In [ ]:
# обучаем базовые алгоритмы на всей обучающей выборке
ml_pipe_1.fit(train, y_train)
ml_pipe_2.fit(train, y_train)
ml_pipe_3.fit(train, y_train)
ml_pipe_4.fit(train, y_train)

In [ ]:
# получаем ответы (вероятности) базовых алгоритмов, обученных
# на всей обучающей выборке, для тестовой выборки
test_m_1_probs = pd.DataFrame(ml_pipe_1.predict_proba(test)[:,1], columns=['m_1_probs'])
test_m_2_probs = pd.DataFrame(ml_pipe_2.predict_proba(test)[:,1], columns=['m_2_probs'])
test_m_3_probs = pd.DataFrame(ml_pipe_3.predict_proba(test)[:,1], columns=['m_3_probs'])
test_m_4_probs = pd.DataFrame(ml_pipe_4.predict_proba(test)[:,1], columns=['m_4_probs'])

In [ ]:
# конкатенируем по оси столбцов всю обучающую выборку с ответами 
# базовых алгоритмов для проверочных блоков
train_stack_df = pd.concat([train, m_1_probs],   axis=1)
train_stack_df = pd.concat([train_stack_df, m_2_probs],   axis=1)
train_stack_df = pd.concat([train_stack_df, m_3_probs],   axis=1)
train_stack_df = pd.concat([train_stack_df, m_4_probs],   axis=1)

In [ ]:
train_stack_df

In [ ]:
# конкатенируем по оси столбцов всю обучающую выборку с ответами 
# базовых алгоритмов для проверочных блоков
#train['m_1_probs'] = t_m1
test_stack_df = pd.concat([test, test_m_1_probs],   axis=1)
test_stack_df = pd.concat([test_stack_df, test_m_2_probs],   axis=1)
test_stack_df = pd.concat([test_stack_df, test_m_3_probs],   axis=1)
test_stack_df = pd.concat([test_stack_df, test_m_4_probs],   axis=1)

In [ ]:
test_stack_df

In [ ]:
# создаем список количественных признаков в новом пространстве признаков
num_columns = train_stack_df.select_dtypes(exclude='object').columns.tolist()
# создаем списки категориальных признаков в новом пространстве признаков
cat_columns = train_stack_df.select_dtypes(include='object').columns

In [ ]:
num_columns, cat_columns

In [ ]:
# создаем конвейеры финальной модели 
# трансформеры
cat_pipe_final = Pipeline([('enc', OneHotEncoder(sparse=False, handle_unknown='ignore'))])
num_pipe_final = Pipeline([('scal', None)]) #
# создаем список трехэлементных кортежей с преобразованиями для определенного типа признаков
transformers_final = [('numf', num_pipe_final, num_columns),
                      ('catf', cat_pipe_final, cat_columns)]
# передаем список трансформеров в ColumnTransformer
transformer_final = ColumnTransformer(transformers=transformers_final)
# задаем итоговый конвейер
ml_pipe_final = Pipeline([('trf', transformer_final), 
                        ('mlf', LGBMClassifier(**final_estimator_params))])

In [ ]:
ml_pipe_final.fit(train_stack_df, y_train)

In [ ]:
# вычисляем вероятности
final_proba = ml_pipe_final.predict_proba(test_stack_df)[:, 1]

In [ ]:
if TUNING_MODE:
    # смотрим значение AUC на тестовой выборке
    print('Значение AUC на тестовой выборке: {}'.format(roc_auc_score(y_test, final_proba)))

In [ ]:
submission = pd.read_csv(input_path / ('sample_submission.' + d_ext), index_col='id')
####### и формируем посылку
submission[TARGET] = final_proba
if  WHERE_PLACE == "kaggle":
    submission.to_csv(NAME_OUTPUT_FILE+'.'+d_ext)
else:    
    submission.to_csv(input_path / (NAME_OUTPUT_FILE+'.'+d_ext))

submission

In [ ]:
end_time = time()
time_taken = end_time - start_time # time_taken is in seconds
hours, rest = divmod(time_taken,3600)
minutes, seconds = divmod(rest, 60)
print('Duration: {} hours {} minutes {} seconds.'.format(hours, minutes, seconds))
print('\n\nВремя окончания работы скрипта: {}\n***********************\n'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")))
with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
    print('\n\nВремя окончания работы скрипта: {}\n***********************\n'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)